# CMFGEN

Database from John Hillier’s CMFGEN, a radiative transfer code designed to solve the radiative transfer and statistical equilibrium equations in spherical geometry.

<div class="alert alert-info">

**Note:**
    
In this example, the data was downloaded from the [CMFGEN website](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) and extracted to the `/tmp/atomic` folder.
    
</div>



## Parsers

The CMFGEN parsers retrieves data from text files preserving its original form (no unit conversions) and stores it in DataFrames. Currently, just `osc`, `col` and `pho` files are supported.


### Levels, Lines and Collisions

Energy levels and spectral lines are stored in the `osc` files, while collisions strengths are provided by the `col` files.

In [1]:
from carsus.io.cmfgen import CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser, CMFGENCollisionalStrengthsParser

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


In [2]:
si2_lvl = CMFGENEnergyLevelsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [3]:
si2_osc = CMFGENOscillatorStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [4]:
si2_col = CMFGENCollisionalStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_col')

Header information is stored in the `header` attribute and the DataFrame in the `base` attribute:

In [5]:
si2_col.header

{'Number of transitions': '406',
 'Number of T values OMEGA tabulated at': '14',
 'Scaling factor for OMEGA': '1.0',
 'Value for OMEGA if f=0': '0.1'}

In [6]:
si2_lvl.base

,label,g,E(cm^-1),10^15 Hz,eV,Lam(A),ID,ARAD,C4,C6
0,3s2_3p_2Po[1/2],2.0,0.00,3.95241,16.346,758.505847,1,0.000000e+00,-5.290000e-17,7.470000e-33
1,3s2_3p_2Po[3/2],4.0,287.24,3.94380,16.310,760.162036,2,0.000000e+00,-5.300000e-17,7.460000e-33
2,3s_3p2_4Pe[1/2],2.0,42824.29,2.66857,11.036,1123.420681,3,1.222000e+04,-5.340000e-17,7.490000e-33
3,3s_3p2_4Pe[3/2],4.0,42932.62,2.66532,11.023,1124.789552,4,2.110000e+03,-5.340000e-17,7.490000e-33
4,3s_3p2_4Pe[5/2],6.0,43107.91,2.66007,11.001,1127.011617,5,3.246000e+03,-5.350000e-17,7.490000e-33
...,...,...,...,...,...,...,...,...,...,...
152,3s_3p(3Po)4p_4Pe[3/2],4.0,134079.00,-0.06718,-0.278,-44625.724052,153,1.159000e+08,-1.500000e-16,3.080000e-32
153,3s_3p(3Po)4p_4Pe[5/2],6.0,134213.63,-0.07122,-0.295,-42096.577969,154,1.167000e+08,-1.500000e-16,3.090000e-32
154,3s_3p(3Po)4p_4Se[3/2],4.0,134905.28,-0.09195,-0.380,-32603.663348,155,9.275000e+07,-9.750000e-17,2.770000e-32
155,3p3_2Do[5/2],6.0,135297.30,-0.10370,-0.429,-28908.752414,156,2.437000e+09,4.200000e-17,1.450000e-32


### Photoionization Cross-sections

Photoionization cross-sections are provided by the `pho` files.

In [7]:
from carsus.io.cmfgen import CMFGENPhoCrossSectionsParser

In [8]:
si2_cross_sections = CMFGENPhoCrossSectionsParser('/tmp/atomic/SIL/II/16sep15/phot_nahar_A')

In [9]:
si2_cross_sections.header

{'Date': '05-Sep-2015 16:54:54',
 'Number of energy levels': '93',
 'Number of photoionization routes': '2',
 'Screened nuclear charge': '2.0',
 'Final state in ion': '3s2_1Se/3s21Se',
 'Excitation energy of final state': '0.0',
 'Statistical weight of ion': '1.0',
 'Sigma used for Gaussian smoothing': '0.0D0',
 'Cross-section unit': 'Megabarns',
 'Split J levels': 'False',
 'Total number of data pairs': '95107',
 'Configuration name': '3s2_3p_2Po',
 'Type of cross-section': '20',
 'Number of cross-section points': '1487'}

In this case, `base` is a list containing many DataFrames. Also, each individual DataFrame contains information relative to that specific target under the attribute `attr`.

In [10]:
type(si2_cross_sections.base)

list

In [11]:
len(si2_cross_sections.base)

93

In [12]:
si2_cross_sections.base[0]

,energy,sigma
0,1.000000,1.414000
1,1.000333,1.405000
2,1.000666,1.397000
3,1.000999,1.388000
4,1.001332,1.379000
...,...,...
1482,17.034180,0.001255
1483,18.825680,0.000929
1484,20.805590,0.000689
1485,22.993730,0.000510


In [13]:
si2_cross_sections.base[0].attrs

{'Configuration name': '3s2_3p_2Po',
 'Type of cross-section': 20,
 'Number of cross-section points': 1487}

### Batch Convert Files to HDF5

To convert multiple CMFGEN files to the HDF5 format import the `hdf_dump` function.

<div class="alert alert-info">

**Note:**
    
This feature has been temporary disabled.
    
</div>

Required parameters are `cmfgen_dir`, `patterns` and `parser`, while `chunk_size` and `ignore_patterns` are optional.

## CMFGENReader

The `CMFGENReader` provides processed `levels`, `lines`, `ionization_energies`, `cross_sections` and `collisions` tables to work with. You can provide temperatures for the `collisions` dataframe using the `temperature_grid` option. 

In [14]:
from carsus.io.cmfgen import CMFGENReader

In [15]:
cmfgen_reader = CMFGENReader.from_config('Si 0-1', '/tmp/atomic', ionization_energies=True, cross_sections=True, collisions=True, drop_mismatched_labels=True)

[      carsus.io.cmfgen.base][WARNING] - Selecting H 0 from CMFGEN (required to ingest cross-sections). (base.py:557)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for H 0. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 0. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 1. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for H 0. (base.py:854)


...

[      carsus.io.cmfgen.base][   INFO] - Dropping collisional ionization data. (base.py:1052)


[      carsus.io.cmfgen.base][   INFO] - Entries having label(s): 3s2_3p2_3Pe, 3s_3p3_5So will be dropped for ion: (14, 0). (base.py:1058)


[      carsus.io.cmfgen.base][   INFO] - Filling in 1 values for ion (1, 0) that are outside the tabulated temperature range using the last tabulated value. (base.py:1118)


[      carsus.io.cmfgen.base][   INFO] - Filling in 7 values for ion (14, 0) that are outside the tabulated temperature range using the last tabulated value. (base.py:1118)


[      carsus.io.cmfgen.base][   INFO] - Filling in 4 values for ion (14, 1) that are outside the tabulated temperature range using the last tabulated value. (base.py:1118)


In [16]:
cmfgen_reader.levels

energy     j  \
atomic_number ion_charge level_index                        
1             0          0                 0.000000   0.5   
                         1             82259.096334   3.5   
                         2             97492.300647   8.5   
                         3            102823.892736  15.5   
                         4            105291.651183  24.5   
...                                             ...   ...   
14            1          152          134079.000000   1.5   
                         153          134213.630000   2.5   
                         154          134905.280000   1.5   
                         155          135297.300000   2.5   
                         156          135303.500000   1.5   

                                                      label method  priority  
atomic_number ion_charge level_index                                          
1             0          0                             1___   meas        10  
                         1                             2___   meas        10  
                         2                             3___   meas        10  
                         3                             4___   meas        10  
                         4                             5___   meas        10  
...                                                     ...    ...       ...  
14            1          152          3s_3p(3Po)4p_4Pe[3/2]   meas        10  
                         153          3s_3p(3Po)4p_4Pe[5/2]   meas        10  
                         154          3s_3p(3Po)4p_4Se[3/2]   meas        10  
                         155                   3p3_2Do[5/2]   meas        10  
                         156                   3p3_2Do[3/2]   meas        10  

[680 rows x 5 columns]

In [17]:
cmfgen_reader.lines

energy_lower  \
atomic_number ion_charge level_index_lower level_index_upper                 
1             0          0                 1                          0.00   
                                           2                          0.00   
                                           3                          0.00   
                                           4                          0.00   
                                           5                          0.00   
...                                                                    ...   
14            1          152               156                   134079.00   
                         153               155                   134213.63   
                                           156                   134213.63   
                         154               155                   134905.28   
                                           156                   134905.28   

                                                               energy_upper  \
atomic_number ion_charge level_index_lower level_index_upper                  
1             0          0                 1                   82259.096334   
                                           2                   97492.300647   
                                           3                  102823.892736   
                                           4                  105291.651183   
                                           5                  106632.159810   
...                                                                     ...   
14            1          152               156                135303.500000   
                         153               155                135297.300000   
                                           156                135303.500000   
                         154               155                135297.300000   
                                           156                135303.500000   

                                                                        gf  \
atomic_number ion_charge level_index_lower level_index_upper                 
1             0          0                 1                  8.324000e-01   
                                           2                  1.582000e-01   
                                           3                  5.798000e-02   
                                           4                  2.788000e-02   
                                           5                  1.559800e-02   
...                                                                    ...   
14            1          152               156                1.300000e-05   
                         153               155                3.963000e-06   
                                           156                2.477400e-07   
                         154               155                3.069200e-05   
                                           156                2.697600e-06   

                                                              j_lower  \
atomic_number ion_charge level_index_lower level_index_upper            
1             0          0                 1                      0.5   
                                           2                      0.5   
                                           3                      0.5   
                                           4                      0.5   
                                           5                      0.5   
...                                                               ...   
14            1          152               156                    1.5   
                         153               155                    2.5   
                                           156                    2.5   
                         154               155                    1.5   
                                           156                    1.5   

                                                      

In [18]:
cmfgen_reader.cross_sections

energy     sigma
atomic_number ion_charge level_index                      
1             0          0              0.999466  6.310022
                         0              1.015765  6.042398
                         0              1.032328  5.786125
                         0              1.049162  5.540721
                         0              1.066271  5.305726
...                                          ...       ...
14            1          154         -263.073454  0.000000
                         155           -0.031522  0.000000
                         155           -3.152219  0.000000
                         156           -0.031579  0.000000
                         156           -3.157868  0.000000

[385466 rows x 2 columns]

In [19]:
cmfgen_reader.ionization_energies

atomic_number  ion_charge
1              0             13.598434
14             0              8.151683
               1             16.345846
Name: ionization_energy, dtype: float64

The values in the collisions dataframe are thermally-averaged effective collision strengths divided by the statistical weights of the lower levels. Please see Eq. A1 and A2 of <cite data-cite="2004ApJ...609.1181P">Przybilla & Butler, 2004 </cite> for definitions. More information about the `collisions` table is stored inside the `collisional_metadata` attribute. 

In [20]:
cmfgen_reader.collisions

0   \
atomic_number ion_charge level_index_lower level_index_upper              
1             0          0                 1                   0.331250   
                                           2                   0.096550   
                                           3                   0.008550   
                                           4                   0.011700   
                                           5                   0.006550   
...                                                                 ...   
14            1          25                29                 39.000000   
                                           30                 13.950000   
                         26                29                  3.550000   
                                           30                 44.833333   
                         29                30                 17.900000   

                                                                     1   \
atomic_number ion_charge level_index_lower level_index_upper              
1             0          0                 1                   0.331250   
                                           2                   0.096550   
                                           3                   0.008550   
                                           4                   0.011700   
                                           5                   0.006550   
...                                                                 ...   
14            1          25                29                 39.000000   
                                           30                 13.950000   
                         26                29                  3.550000   
                                           30                 44.833333   
                         29                30                 17.900000   

                                                                     2   \
atomic_number ion_charge level_index_lower level_index_upper              
1             0          0                 1                   0.331250   
                                           2                   0.096550   
                                           3                   0.008550   
                                           4                   0.011700   
                                           5                   0.006550   
...                                                                 ...   
14            1          25                29                 39.000000   
                                           30                 13.950000   
                         26                29                  3.550000   
                                           30                 44.833333   
                         29                30                 17.900000   

                                                                     3   \
atomic_number ion_charge level_index_lower level_index_upper              
1             0          0                 1                   0.331250   
                                           2                   0.096550   
                                           3                   0.008550   
                                           4                   0.011700   
                                           5                   0.006550   
...                                                                 ...   
14            1          25                29                 43.500000   
                                           30                 15.300000   
                         26                29                  3.866667   
                                           30                 51.333333   
                         29                30                 19.350000   

                                                                     4   \
atomic_number ion_charge level_index_lower level_index_upper              
1             0

In [21]:
cmfgen_reader.collisional_metadata

temperatures    [100, 2000, 2500, 4000, 5000, 6300, 10000, 126...
dataset                                                  [cmfgen]
info            The dataframe values are thermally-averaged ef...
dtype: object